In [1]:
import random
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import string
import pyphen
import os

In [2]:
def warp_image(img, gridsize=(26, 26), deviation=1.7):
    """
    Применяет искажение к изображению с использованием случайной сетки.
    """
    (w, h) = img.size

    num_x = w // gridsize[0] + 1
    num_y = h // gridsize[1] + 1

    # Генерация случайной матрицы искажения
    mat = np.random.normal(scale=deviation, size=(num_y + 1, num_x + 1, 2))

    mesh = []
    for x in range(num_x):
        for y in range(num_y):
            target = (x * gridsize[0], y * gridsize[1],
                      (x + 1) * gridsize[0], (y + 1) * gridsize[1])

            nw_y = y * gridsize[1] + mat[y, x, 0]
            nw_x = x * gridsize[0] + mat[y, x, 1]

            sw_y = (y + 1) * gridsize[1] + mat[y + 1, x, 0]
            sw_x = x * gridsize[0] + mat[y + 1, x, 1]

            se_y = (y + 1) * gridsize[1] + mat[y + 1, x + 1, 0]
            se_x = (x + 1) * gridsize[0] + mat[y + 1, x + 1, 1]

            ne_y = y * gridsize[1] + mat[y, x + 1, 0]
            ne_x = (x + 1) * gridsize[0] + mat[y, x + 1, 1]

            source = (nw_x, nw_y, sw_x, sw_y, se_x, se_y, ne_x, ne_y)

            mesh.append((target, source))

    # Применение искажения
    img_transformed = img.transform(
        img.size,
        method=Image.MESH,
        data=mesh,
        fillcolor=255)

    return img_transformed

In [3]:
# Функция для переноса строк
def wrap_text(text, font, max_width,lines_per_page):
    dic = pyphen.Pyphen(lang='ru')
    lines = []
    output=[]
    text = [i.split() for i in text.split('\n')]
    for words in text:
        while words:
            words[0]=words[0].strip()
            space=random.randint(0,1)*' '
            line = ''+space
            while words and font.getlength(line + words[0] + ' ') <= max_width:
                line += words.pop(0) + ' '
            if words!=[]:
                hyphenated_word = dic.inserted(words[0]).split('-')
                while hyphenated_word:
                    flag=True
                    if len(hyphenated_word)==2:
                        if len(hyphenated_word[1])<4:
                            flag=False
                    if font.getlength(line + hyphenated_word[0] + '-')<= max_width and flag:
                        line += hyphenated_word.pop(0)
                    else:
                        if not words[0]=="".join(hyphenated_word):
                            line+='-'
                            words[0]="".join(hyphenated_word)
                        break
            lines.append(line)
            if len(lines)==lines_per_page:
                output.append(lines)
                lines=[]
    if len(lines)<lines_per_page:
        while len(lines)<lines_per_page:
            lines.append([])
        output.append(lines)
    return output

In [4]:
# Настройка шрифта
font_path = "eskal_font4you.ttf"  # Замените на путь к вашему шрифту
font = ImageFont.truetype(font_path, size=400,layout_engine=ImageFont.Layout.BASIC)
width,height=(6992, 9921) # https://pixelcalculator.com/en
draw_lines=True # Рисовать линии строк?

endings=string.whitespace+string.punctuation
os.makedirs('output',exist_ok=True)

In [18]:
# Текст
text = open('text.txt').read() # Переносы в тексте учитываются
# text=""

x_start, y_start = (width//13,height//45)  # Начальные координаты
padding=x_start
kerning = -width//400  # Отрицательное значение уменьшает расстояние между буквами
line_spacing = 0.97  # Межстрочный интервал
max_offset_step = font.size//100
#max_offset_total = 10
max_angle = 5
max_line_offset = 0
#whitespace_end_only=True
lines_per_page=24

line_color = "grey"  # Цвет линии
line_thickness = height//1000  # Толщина линии

gridsize = (font.size//2, font.size//2)  # Размер сетки для искажения
deviation = font.size//20  # Сила искажения

# Перенос строк
wrapped_text = wrap_text(text, font, width*0.95-x_start,lines_per_page)

for index,page in enumerate(wrapped_text):
    if index%2==0:
        x_start=padding
    else:
        x_start=x_start//10
    print(page)
    x, y = x_start, y_start
    # Создаём холст
    image = Image.new("RGB", (width,height), color='white')
    #image=Image.open('a5_1200.jpg')
    draw = ImageDraw.Draw(image)
    for c,line in enumerate(page):
        if not len(page)-c==1 and draw_lines:
            draw.line([(0, int(y + font.size*0.8)), (width, int(y + font.size*0.8))], fill=line_color, width=line_thickness)

        for i in range(len(line)):
            char = line[i]
            left, top, right, bottom = font.getbbox(char)  # Извлекаем ширину и высоту
            char_width, char_height = (right - left, bottom - top)

            if char == ' ':
                char_width = random.randint(int(char_width*0.7),int(char_width*1.3))

            # Смещение и вращение для каждой буквы
            #offset_x = random.randint(-max_offset_step, max_offset_step)  # Случайное смещение по X
            offset_x = 0  # Случайное смещение по X
            offset_y = random.randint(-max_offset_step, max_offset_step)  # Случайное смещение по Y
            angle = random.randint(-max_angle, max_angle)  # Случайное вращение

            # Создаём изображение для буквы
            char_image = Image.new("RGBA", (int(char_width*1.4), int(char_height*1.4)), (255, 0, 0, 0))
            char_draw = ImageDraw.Draw(char_image)
            char_draw.text((-left+char_width//5, -top+char_height//5), char, font=font, fill="black")

            # Поворот буквы
            char_image = char_image.rotate(angle, resample=Image.BICUBIC, expand=True)

            # Применение искажения сетки
            char_image = warp_image(char_image, gridsize=gridsize, deviation=deviation)

            # Вставляем букву в основное изображение
            image.paste(char_image, (x + offset_x-char_width//5, y + offset_y + top-char_height//5), char_image)

            # Коррекция ширины для конкретных символов
            if char in 'ПбйТГ':
                char_width = int(char_width*0.7)
            if char in 'б':
                char_width = int(char_width*0.7)
            if line[min(i + 1, len(line) - 1)] in "рд":
                char_width = int(char_width*0.5)
            if line[min(i + 1, len(line) - 1)] in "уз":
                char_width = int(char_width*0.7)

            # Смещаем координату X с учётом кернинга
            x += char_width + kerning

        # Переход на новую строку
        x = x_start
        line_height = int(font.size * line_spacing)  # Высота строки с учетом межстрочного интервала
        y += line_height + random.randint(-max_line_offset, max_line_offset)  # Добавляем случайное отклонение
    image.save(os.path.join('output',f'{index}.png'),format='png')
    # Отображаем результат
    #display(image)

[' Стандартизация — деятельность по ', 'установлению норм, правил и характери-', 'стик с целью обеспечения безопасности ', 'продукции, работ и услуг для окружа-', 'ющей среды, жизни, здоровья и имуще-', ' ства; технической и информационной ', 'совместимости, взаимозаменяемости про-', ' дукции; качества продукции, работ и ', ' услуг в соответствии с уровнем науки ', 'и техники; единства измерений; эко-', ' номии ресурсов; безопасности объектов с ', 'учетом риска природных и техногенных ', 'катастроф; обороноспособности и мобили-', 'зационной готовности страны. ', 'Стандартизация направлена на дости-', 'жение оптимальной степени упорядо-', 'чения в определенной области путем ', 'установления положений для всеобщего ', ' и многократного использования. ', ' Объект стандартизации — продукция, ', ' процесс или услуга, подвергшиеся или ', ' подлежащие стандартизации. Область ', 'стандартизации — совокупность взаимо-', ' связанных объектов стандартизации. ']
[' Основные принципы стандартизации

In [5]:
def combine_photos(photo_paths, output_path, line_color="black", line_thickness=2):
    """
    Совмещает фотографии по парам в альбомной ориентации с вертикальной линией на границе.

    :param photo_paths: Список путей к портретным фотографиям.
    :param output_path: Путь для сохранения объединенных фотографий.
    :param line_color: Цвет разделительной линии (по умолчанию: черный).
    :param line_thickness: Толщина разделительной линии (по умолчанию: 2 пикселя).
    """

    photo_paths.extend([None for _ in range(4-len(photo_paths)%4)])
    paired_photos=[]
    for i in range(0,len(photo_paths),4):
        paired_photos.append((photo_paths[i],photo_paths[i+2]))
        paired_photos.append((photo_paths[i+3],photo_paths[i+1]))

    for idx, (photo1_path, photo2_path) in enumerate(paired_photos):
        if not photo1_path:
            photo1 = Image.open(f'{photo_folder}/blank.png')
        else:
            photo1 = Image.open(photo1_path)

        if not photo2_path:
            photo2 = Image.open(f'{photo_folder}/blank.png')
        else:
            photo2 = Image.open(photo2_path)

        # Проверяем ориентацию
        if photo1.size[0] > photo1.size[1] or photo2.size[0] > photo2.size[1]:
            raise ValueError("Фотографии должны быть в портретной ориентации.")

        # Вычисляем размеры нового изображения
        width = photo1.size[0] + photo2.size[0] + line_thickness
        height = max(photo1.size[1], photo2.size[1])

        # Создаем новое изображение
        combined_image = Image.new("RGB", (width, height), "white")
    
        # Вставляем фотографии
        combined_image.paste(photo1, (0, 0))
        combined_image.paste(photo2, (photo1.size[0] + line_thickness, 0))

        # Рисуем разделительную линию
        draw = ImageDraw.Draw(combined_image)
        line_x = photo1.size[0]
        draw.rectangle([(line_x, 0), (line_x + line_thickness - 1, height)], fill=line_color)
        
        # Сохраняем результат
        output_file = os.path.join(output_path, f"combined_{idx + 1}.png")
        combined_image.save(output_file)
        #display(combined_image)

# Пример использования
photo_folder = "output"  # Папка с фотографиями
output_folder = "output_merged"  # Папка для сохранения результатов
os.makedirs(output_folder, exist_ok=True)
files=os.listdir(photo_folder)
files.remove('blank.png')
files.sort(key=lambda x: int(x.removesuffix('.png')))
# Список фотографий
photos = [os.path.join(photo_folder, f) for f in files]

combine_photos(photos, output_folder)


In [ ]:
# Настройка шрифта
font_path = "eskal_font4you.ttf"  # Замените на путь к вашему шрифту
font = ImageFont.truetype(font_path, size=50)

# Алфавит (можете изменить набор символов, если необходимо)
characters = 'абвгдеёжзиклмнопрстуфхцчшщъэьюя'.upper()+'абвгдеёжзиклмнопрстуфхцчшщъэьюя'

# Генерация и вывод изображений для каждого символа
for char in characters:
    # Определяем размеры символа
    left, top, right, bottom = font.getbbox(char)  # Извлекаем ширину и высоту
    char_width, char_height=(right-left,bottom-top)
    # Создаём изображение, равное размеру символа
    char_image = Image.new("RGBA", (char_width, char_height), (255, 255, 255, 0))
    warped_image = warp_image(char_image, gridsize=(30, 30), deviation=10)
    char_draw = ImageDraw.Draw(warped_image)

    # Рисуем символ
    char_draw.text((-left, -top), char, font=font, fill="black")

    # Выводим символ через display
    print(f"Символ: '{char if char.strip() else 'space'}'",left, top, right, bottom)
    display(warped_image)
